# Competitiveness of Taiwan Cycles (HS 871200)

In [1]:
import pandas as pd

path = '//172.26.1.102/dstore/uncomtrade/annual/type-C_r-ALL_ps-2015_freq-A_px-HS_pub-20170213_fmt-csv_ex-20170226.csv'
df = pd.read_csv(path, dtype={'Commodity Code': str})

In [2]:
product = '871200'
cycle = df.loc[(df['Aggregate Level'] == 6) & (df['Trade Flow'] == 'Import') &
               (df['Partner'] != 'World') & (df['Commodity Code'] == product),
               ['Reporter', 'Partner', 'Commodity Code', 'Commodity', 'Trade Value (US$)']]
cycle.columns = ['reporter', 'partner', 'code', 'commodity', 'val']
# Total import of each reporter
cycle['world'] = cycle.groupby('reporter')['val'].transform('sum')
# Market share of each partner
cycle['share'] = cycle['val'] / cycle['world'] * 100
# Rank of each partner
cycle['rank'] = cycle.groupby('reporter')['val'].rank(ascending=False, method='min')

In [3]:
cycle[cycle['partner'] == 'Other Asia, nes'].drop('partner', axis=1).sort_values(
    ['rank', 'share'], ascending=[True, False]).rename(
    columns={'reporter' : 'reporter 進口國',
             'code'     : 'code 產品代碼',
             'commodity': 'commodity 產品名稱',
             'val'      : 'import_tw ($USD) 自臺進口額',
             'world'    : 'import_world ($USD) 自全球進口額',
             'share'    : 'market_share 臺灣自占率',
             'rank'     : 'rank 臺灣排名'
            }).to_csv('cycle_raw.csv', sep=',', index=False)